Instead of building separate models for claim frequency and claim severity, the Tweedie distribution can be used to model pure premium directly. The Tweedie family of distributions are Exponential Dispersion Models with variance function $V(\mu) = \mu^{p}$ where $1 \lt p \lt 2$. 

However, the Tweedie distribution has an important limitation: Buried in the model parameterization is the assumption of positive correlation between claim frequency and severity. Put another way, as commonly implemented within various GLM frameworks, the Tweedie distribution assumes that each covariate impacts frequency and severity in the same direction.

Tweedie is a compund Poisson-gamma process, where claim frequency is assumed to follow Poisson distribution and claim severity assumed to follow a gamma distribution (conditional on the occurrence of a claim). In the original [Smyth and Jorgensen paper](https://www.casact.org/sites/default/files/old/astin_vol32no1_143.pdf), the Tweedie distribution is parameterized as follows:

$$
\mu = \lambda \cdot \alpha \cdot \theta; \hspace{.75em} p = \frac{\alpha + 2}{\alpha + 1}; \hspace{.75em} \phi = \frac{\lambda^{1 - p} \cdot (\alpha \cdot \theta)^{2 - p}}{2 - p}
$$

Where:

- $\lambda$ is the mean of the underlying Poisson distribution.
- $\alpha$ is the gamma distribution shape parameter.
- $\theta$ is the gamma distribution scale parameter.
- $p$ is constant across the dataset, specified by the user. Since $\alpha \gt 0$, $1 \lt p \lt 2$.


Because $p$ is constant, we have:

$$
\phi \sim \frac{(\alpha \cdot \theta)^{2 - p}}{\lambda^{p-1}}
$$

$\phi$ is proportional to the mean severity raised to some positive power divided by the mean frequency raised to some positive power. If $\phi$ is assumed to be constant across the dataset (as is the default assumption in many popular libraries such as scikit-learn and statsmodels), any factor that increases mean severity must also increase mean frequency. To ensure that $\phi$ remains constant, mean frequency and mean severity must move in the same direction. However, the assumption that frequency and severity move in the same direction is often unrealistic. Think of how auto claims are usually distributed: A large number of low severity claims, along with fewer of high severity. 


The Tweedie distribution can still be used in situations in which frequency and severity move in opposite directions, as long as $\phi$ and $\mu$ are modeled together. This is referred to as a double GLM, and is outlined in the Smyth and Jorgensen paper. If the dispersion parameter is allowed to vary across the dataset, the assumption of positive association between frequency and severity no longer applies.

Fitting a Tweedie double GLM can be accomplished as follows:

1. Using pure premium as the target, fit a Tweedie GLM for the mean. The weight is exposures divided by $\phi$, where initially $\phi = 1$ for all inputs.

2. Calculate the weighted deviance for each record. The sum of these unit deviances equals the total deviance for the initial model for the mean.

3. Fit a gamma GLM for dispersion. The target variable is the unit deviance with each record given equal weight. The predictions from this model represent the new per observation dispersion.

4. Re-fit the GLM for the mean from step 1., this time using the dispersion parameters obtained in step 3. The weight for this GLM is exposure divided by the new dispersion parameters.

This is straightforward to implement, and is left as an exercise for the reader. For R users, the [dglm package](https://cran.r-project.org/web/packages/dglm/dglm.pdf) is available. 

I recall a few years back it wasn't possible to set `p` to a variable when using the dglm package; `p` had to be hard-coded with a scalar value, which was inconvenient if you wanted to evaluate a number of Tweedie models with different `p`s in an iterative fashion. A work-around was proposed [here](https://stackoverflow.com/questions/28810479/package-dglm-in-r), but the package creator may have since incorporated the update into the codebase. 
